# PyPika Insight Building Demo

## PyPika Imports

In [14]:
from pypika import (
    Query,
    Table,
    Tables,
    Field,
    AliasedQuery,
    Query,
    CustomFunction,
    Order,
    Database,
    Schema,
    Case,
    Criterion,
    analytics as an,
    functions as fn
)

from mimo_custom import custom_functions as cf
from mimo_custom import variable_list as vl

## Variables

In [15]:
merchant_list = [ 'BCB PAYMENTS LTD',
      'BINANCE',
      'BITCOIN',
      'BITMAX',
      'BITPANDA',
      'BITSTAMP',
      'CB PAYMENT',
      'CLEARBANK',
      'COIN BASE',
      'COINBASE',
      'COINBURP',
      'COINCORNER',
      'COINFLOOR',
      'COINIFY',
      'COINMETRO',
      'COINPASS',
      'CRYPCFPS',
      'CRYPRO',
      'CRYPTO',
      'CRYPTOCOM',
      'CRYPTODOTCOM',
      'ETORO',
      'FIAT WALLET',
      'FORIS DAX',
      'GEMINI EUROPE',
      'KENSPECKLE',
      'KRYPTKE',
      'KUCOIN',
      'LUNA MONEY',
      'LUNO ',
      'NEXO',
      'KRACKEN',
      'SKRILL',
      'LUNO MONEY',
      'MOON PAY',
      'PAYWARD',
      'SBORG',
      'SOLIDI ',
      'SPREADEX',
      'SWISSBORG',
      'UPHOLD EUROPE',
      'UPHOLD ',
      'CB',
      'CRO']

## Initialise Tables

In [16]:
# schema = Schema("consumption_db")

# transactions_table = Table("transactions_v", schema=schema).as_("base")
# pa_table = Table("parties_accounts_relations_v",schema=schema)

transactions_table = Table("transactions_v_incl_cc").as_("base")
pa_table = Table("parties_accounts_relations_v")

## Parties Accounts Relations

In [17]:
pa_mapping = Query.from_(pa_table).select("*").as_("pa_mapping")

# pa_mapping.get_sql(quote_char=None)

## Attribute Lookup

In [18]:
# my_db = Database('consumption_db')

attr_lkp_acct = Table("attribute_lookup_master")

attr_lkp_acct = Query.from_(attr_lkp_acct).select(attr_lkp_acct.lookup_value).where( (attr_lkp_acct.insight_id == "3465-CRYPTO-SCAMS-PG") & (attr_lkp_acct.lookup_key == "SORTCODE_ACCOUNT"))

# attr_lkp_acct.get_sql(quote_char=None)

In [19]:
# my_db = Database('consumption_db')

attr_lkp = Table("attribute_lookup_master")

attr_lkp = Query.from_(attr_lkp).select(attr_lkp.lookup_value).where((attr_lkp.insight_id == "3465-CRYPTO-SCAMS-PG") & (attr_lkp.lookup_key == "SORTCODE"))

# attr_lkp.get_sql(quote_char=None)

## List Filters

In [20]:
# Where conditions for Merchant List
merchant_filter = vl.NarrativeCriterions.generate_criterion("Upper",transactions_table.clean_narrative, "any", merchant_list)

# Where conditions for Attribute LookUp Table 
alt_filter_1 = fn.Upper(fn.Concat(cf.Lpad((fn.Trim(transactions_table.fnd_xfer_opp_sort_cd)),6,0),cf.Lpad((fn.Trim(transactions_table.fnd_xfer_opp_acct_num)),8,0))).isin(attr_lkp_acct)
alt_filter_2 = fn.Upper(cf.Lpad((fn.Trim(transactions_table.fnd_xfer_opp_sort_cd)),6,0)).isin(attr_lkp)


## Join Transactions and Parties Accounts Relations

In [21]:
rank = (
    an.Rank()
    .over(pa_mapping.prophet_party_id)
    .orderby(transactions_table.transaction_booking_timestamp, order=Order.desc)
    .orderby(transactions_table.amount, order=Order.desc)
    .orderby(transactions_table.merchant_brand_name, order=Order.asc)
    .orderby(transactions_table.clean_narrative, order=Order.asc)
    .orderby(transactions_table.prophet_transaction_id, order=Order.desc)
    .as_("_rank")
)

trn_pa_join = (
    Query.from_(transactions_table).inner_join(pa_mapping)
    .on_field("prophet_account_id").select(
    pa_mapping.prophet_party_id.as_("prophet_party_id_a"),
    pa_mapping.brand.as_("brand_a"),
    transactions_table.transaction_booking_timestamp,
    transactions_table.amount,
    transactions_table.merchant_brand_name,
    transactions_table.clean_narrative,
    rank)
    .where(cf.ToDate(transactions_table.transaction_booking_timestamp)[cf.DateSub(cf.ToDate('2021-05-22T12:00:00.000'),7):cf.ToDate('2021-05-29T12:00:00.000')])
    .where(cf.UnixTimestamp(transactions_table.record_creation_timestamp,{0}) >= cf.UnixTimestamp('2021-05-22T12:00:00.000',{0}))
    .where(cf.UnixTimestamp(transactions_table.record_creation_timestamp,{0}) < cf.UnixTimestamp('2021-05-29T12:00:00.000',{0}))
    .where((transactions_table.pos_refund_flag != 1) | (transactions_table.pos_refund_flag.isnull()))
    .where(-(transactions_table.amount)>0)
    .where(merchant_filter | alt_filter_1 | alt_filter_2)
).as_("sub_query")

# trn_pa_join.get_sql(quote_char=None)

## Sub Query

In [22]:
sub_query = Query.from_(trn_pa_join).select("*").where(trn_pa_join._rank==1)

# secondary_aggregate.get_sql(quote_char=None)

## Final Query

In [23]:
parent_query = (Query
            .with_(sub_query, "sq")
            .from_(AliasedQuery("sq"))
            .select(
                    sub_query.prophet_party_id_a.as_("prophet_party_id"),
                    sub_query.brand_a.as_("brand"),
                    fn.Cast(None,"String").as_("prophet_account_id"),
                    cf.reflect("java.util.UUID", "randomUUID").as_("insight_id"),
                    cf.FromUnixTime(cf.UnixTimestamp(),{0}).as_("event_timestamp"),
                    fn.Cast(None,"String").as_("transaction_record_creation_timestamp"),
                    fn.Cast(None,"String").as_("transaction_booking_timestamp"),
                    fn.Cast(None,"String").as_("transaction_value_timestamp"),
                    Case()
                       .when(sub_query.merchant_brand_name.isnull(), sub_query.clean_narrative)
                       .else_(sub_query.merchant_brand_name).as_('insight_attributes'),
                    fn.Cast("0.0.1","String").as_("user_story_version"),
                    fn.Cast("HttpLIVYOperator","String").as_("created_by"),
                    cf.FromUnixTime(cf.UnixTimestamp(),{0}).as_("record_creation_timestamp"),
                    fn.Cast(None,"String").as_("data_source"),
                    fn.Cast("3465-Crypto-Scams-Pg","String").as_("user_story_id")
))

In [24]:
final_query = parent_query.get_sql(quote_char=None)
final_query.format(""" "yyyy-MM-dd'T'HH:mm:ss.SSS" """)

'WITH sq AS (SELECT * FROM (SELECT pa_mapping.prophet_party_id prophet_party_id_a,pa_mapping.brand brand_a,base.transaction_booking_timestamp,base.amount,base.merchant_brand_name,base.clean_narrative,RANK() OVER(PARTITION BY pa_mapping.prophet_party_id ORDER BY base.transaction_booking_timestamp DESC,base.amount DESC,base.merchant_brand_name ASC,base.clean_narrative ASC,base.prophet_transaction_id DESC) _rank FROM transactions_v_incl_cc base JOIN (SELECT * FROM parties_accounts_relations_v) pa_mapping ON base.prophet_account_id=pa_mapping.prophet_account_id WHERE TO_DATE(base.transaction_booking_timestamp) BETWEEN DATE_SUB(TO_DATE(\'2021-05-22T12:00:00.000\'),7) AND TO_DATE(\'2021-05-29T12:00:00.000\') AND UNIX_TIMESTAMP(base.record_creation_timestamp, "yyyy-MM-dd\'T\'HH:mm:ss.SSS" )>=UNIX_TIMESTAMP(\'2021-05-22T12:00:00.000\', "yyyy-MM-dd\'T\'HH:mm:ss.SSS" ) AND UNIX_TIMESTAMP(base.record_creation_timestamp, "yyyy-MM-dd\'T\'HH:mm:ss.SSS" )<UNIX_TIMESTAMP(\'2021-05-29T12:00:00.000\', "